## テキストの前処理
[janome](https://mocobeta.github.io/janome/)を利用する  
*   インストール 
*   形態素解析
*   品詞によるフィルタリング 
*   [ストップワード](http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt)の除去







In [ ]:
!pip install janome  #インストール

In [ ]:
#形態素解析
from janome.tokenizer import Tokenizer
text = 'なんて美しい風景でしょう！感動した！！'

t = Tokenizer() #何も指定しないと，すべての情報を出力
for token in t.tokenize(text):
  print(token)

print('------------------------')

t = Tokenizer(wakati = True) #wakati = Trueを指定すると分かち書きを行う
result = [t for t in t.tokenize(text)]  
print( result )

In [ ]:
#品詞によるフィルタリング
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, POSStopFilter #POS = PartOfSpeech = 品詞

text = 'なんて美しい風景でしょう！感動した！！'

#フィルタの定義
filters = [] #フィルタなし --> 全部を取り出す
#filters = [ POSKeepFilter( ['名詞', '動詞'] )] #PosKeepFilter：キープする（残す）品詞を指定する
#filters = [ POSStopFilter( '記号' )] #StopFilter：止める（捨てる）品詞を指定する

analyzer = Analyzer(token_filters = filters)
for token in analyzer.analyze(text):
  print(token)  

In [ ]:
#stop word辞書の獲得
!wget http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt .

In [ ]:
# ストップワードの除去
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, POSStopFilter #POS = PartOfSpeech

# stop word辞書の読み込み（作業ディレクトリにJapanese.txtがあることを前提としている）
def load_stopword_dictionary() : 
  with open('Japanese.txt', 'r', encoding='utf-8') as f :
    stopwords = [w.strip() for w in f ]
    stopwords = set(stopwords)
  return stopwords

# 除去関数の定義
def remove_stopwords( words, stopwords ):
  remained_words = [ w for w in words if w not in stopwords]
  return remained_words

stopwords = load_stopword_dictionary()
text = 'なんて美しい風景でしょう！感動した！！'
filters = [ POSKeepFilter( ['名詞'] )] #名詞だけを取り出す
#filters = [ ] 

analyzer = Analyzer(token_filters = filters)
words = [w.surface for w in analyzer.analyze(text)]

words = remove_stopwords(words, stopwords)
print( words )


## カウントベースによる単語ベクトル化
*   準備：関数の定義
*   データ（[羅生門](https://www.aozora.gr.jp/cards/000879/card127.html)）の読み込みと共起行列・PPMI行列の作成

*   類似単語の検索・表示


In [ ]:
#準備：関数の定義

import numpy as np
from janome.analyzer import Analyzer
from janome.tokenfilter import POSKeepFilter, POSStopFilter #POS = PartOfSpeech = 品詞

# load_stopword_dictionary, remove_stopwordsは，先ほどと同じもの
def load_stopword_dictionary() : 
  with open('Japanese.txt', 'r', encoding='utf-8') as f :
    stopwords = [w.strip() for w in f ]
    stopwords = set(stopwords)
  return stopwords

# 除去関数の定義
def remove_stopwords( words, stopwords ):
  remained_words = [ w for w in words if w not in stopwords]
  return remained_words

# 前処理用関数
def preprocess(text, stopwords):
  filters = [ POSKeepFilter( ['名詞', '形容詞', '動詞'] )] #ここは適当に合わせる
  analyzer = Analyzer(token_filters = filters)
  words = [w.surface for w in analyzer.analyze(text)]
  words = remove_stopwords(words, stopwords) #stopword除去

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word

  corpus = np.array( [word_to_id[w] for w in words] )
  return corpus, word_to_id, id_to_word

#共起行列の構築
def create_co_matrix(corpus, vocab_size, window_size=1):
  corpus_size = len(corpus)
  co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

  for idx, word_id in enumerate(corpus):
    for i in range(1, window_size + 1):
      left_idx = idx - i
      if left_idx >= 0:
        left_word_id = corpus[left_idx]
        co_matrix[word_id, left_word_id] += 1

      right_idx = idx + i
      if right_idx < corpus_size:
        right_word_id = corpus[right_idx]
        co_matrix[word_id, right_word_id] += 1

  return co_matrix

#共起行列CをPPMI行列Mへ変換
def ppmi(C, eps = 1e-8):
  M = np.zeros_like(C, dtype=np.float32)
  N = np.sum(C)
  S = np.sum(C, axis=0)
  total = C.shape[0] * C.shape[1]
  cnt = 0

  for i in range(C.shape[0]):
    for j in range(C.shape[1]):
      pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
      M[i, j] = max(0, pmi)

  return M

#ベクトルx,y間のコサイン類似度
def cos_similarity(x, y, eps=1e-8):
  nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
  ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
  return np.dot(nx, ny)

#コサイン類似度上位top件の表示
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
  if query not in word_to_id:
    print('%s is not found' % query)
    return

  print('\n[query] ' + query)
  query_id = word_to_id[query]
  query_vec = word_matrix[query_id]

  vocab_size = len(id_to_word)

  similarity = np.zeros(vocab_size)
  for i in range(vocab_size):
    similarity[i] = cos_similarity(word_matrix[i], query_vec)

  count = 0
  for i in (-1 * similarity).argsort():
    if id_to_word[i] == query:
      continue
    print(' %s: %s' % (id_to_word[i], similarity[i]))

    count += 1
    if count >= top:
      return


In [ ]:
# ファイルの読み込みと共起行列・PPMI行列の作成

#対象データの読み込み
#作業ディレクトリにデータ（rashomon.txt）が配置されていることを前提
with open('rashomon.txt', 'r', encoding='utf-8') as f :
  text = [w.strip() for w in f]
document = ' '.join(text) #対象テキスト

stopwords = load_stopword_dictionary() #stopword辞書の準備

#コーパスの構築
#curpus : documentを単語idに変換したもの
#word_to_id：単語 --> ID 辞書
#id_to_word : ID --> 単語 辞書
(corpus, word_to_id, id_to_word) = preprocess(document, stopwords) 

#共起行列の構築，window_sizeで「文脈」の範囲を指定
co_matrix = create_co_matrix(corpus, len(word_to_id), window_size = 4)
ppmi_matrix = ppmi( co_matrix ) #PPMI行列の構築

print( np.shape( ppmi_matrix ))  #単語数の確認
print( word_to_id ) #単語の確認



In [ ]:
#類似単語を検索してみよう！　結果は微妙？．．
most_similar('老婆', word_to_id, id_to_word, co_matrix)
most_similar('風', word_to_id, id_to_word, co_matrix)
most_similar('胡麻', word_to_id, id_to_word, co_matrix)
most_similar('今日', word_to_id, id_to_word, co_matrix)



## 推論ベースによる単語ベクトル化
*   学習済みモデル（[Facebook公開のfastText](https://fasttext.cc/)）のダウンロード
*   学習済みモデル（[Wikipedia Entity Vectors（w2vモデル）](https://github.com/singletongue/WikiEntVec/)）のダウンロード
*  モデルの読み込み
*  モデルの利用  



In [ ]:
#1.25GBくらいあります（時間のある時に実行しましょう）
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.vec.gz .

In [ ]:
!wget https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.entity_vectors.100d.txt.bz2 .

In [ ]:
#モデルの読み込み：5-10分程度かかります（時間のある時に試しましょう） facebook/wikipedia 任意で選択してください
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('cc.ja.300.vec.gz',binary=False) 
# model = gensim.models.KeyedVectors.load_word2vec_format('jawiki.entity_vectors.100d.txt.bz2',binary=False) 


In [ ]:
#モデルの利用1 単語間類似度の計算
model.similarity('犬','馬') 

In [ ]:
#モデルの利用2 類似語の検索
model.most_similar('科学')  

In [ ]:
#モデルの利用3 類推（ベクトル計算）
# positiveに含まれる単語ベクトルの和 - negativeに含まれる単語ベクトルの和
model.most_similar(positive=['王様','女性'],negative=['男性'], topn=5) #（王様 + 女性 - 男性=??）